# Domain Modeling by Julia

- Author: Joseph Kim <cloudeyes@gmail.com>

In [1]:
import Base.@kwdef
import Dates

## Domain models

In [2]:
@kwdef struct OrderLine
    reference::String
    qty::Int
    sku::String
end;

In [3]:
mutable struct Qty ; value::Int ; end
@kwdef struct Batch
    reference::String
    sku::String
    qty::Qty
    eta::Union{Dates.Date, Nothing} = nothing
    allocations::Array{OrderLine,1} = []
end;

### Equality and ordering for entities

In [103]:
Base.:(==)(x::Batch, y::Batch) = x.reference == y.reference
Base.:(isless)(x::Batch, y::Batch) =
    if x.eta == nothing true
    elseif y.eta == nothing false
    else x.eta < y.eta
end

## Functions on domain model types

**Julia does not have classes**. Instead we define new types and then define methods on those types. Methods are not "owned" by the types they operate on. Instead, a method can be said to belong to a generic function of the same name as the method

- https://stackoverflow.com/a/56352954

In [56]:
function allocate!(batch::Batch, line::OrderLine)
    push!(batch.allocations, line);
end

allocate! (generic function with 1 method)

In [57]:
function availablequantity(batch::Batch)
    batch.qty.value - reduce(+, x.qty for x in b.allocations; init=0)
end

function canallocate(batch::Batch, line::OrderLine) 
    availablequantity(batch) >= line.qty
end;

### Service functions

In [129]:
struct OutOfStock <: Exception ; end

function allocate_batches!(line::OrderLine, batches::Array{Batch,1})
    sorted = sort(filter(b->canallocate(b, line), batches))
    if length(sorted) == 0
        throw(OutOfStock())
    else
        batch = first(sorted)
        allocate!(batch, line)
        return batch.reference
    end
end;

## Tests

### Helper functions

In [134]:
function makebatchandline(sku::String, batchqty::Int, lineqty::Int)
    Batch(reference="batch-001", sku=sku, qty=Qty(batchqty)),
    OrderLine(reference="oder-123", sku=sku, qty=lineqty)
end;

In [135]:
import Test.@test
import Test.@test_throws
import Test.@testset

In [137]:
@testset "Chapter1 Tests" begin
    @testset "Test allocating to a batch reduces the available quantity" begin
        batch = Batch(reference="batch-001", sku="TEST-TABLE", qty=Qty(20))
        line  = OrderLine(reference="order-ref", sku="TEST-TABLE", qty=2)
        @test 20 == availablequantity(batch)
        allocate!(batch, line)
        @test 18 == availablequantity(batch)      
    end
    
    @testset "Test can allocate if available greater than required" begin
        largebatch, smallline = makebatchandline("ELEGANT-LAMP", 20, 2)
        @test canallocate(largebatch, smallline)
    end

    @testset "Test cannot allocate if available smaller than required" begin
        smallbatch, largeline = makebatchandline("ELEGANT-LAMP", 2, 20)
        @test !canallocate(smallbatch, largeline)
    end

    @testset "Test can allocate if available equal to required" begin
        batch, line = makebatchandline("ELEGANT-LAMP", 2, 2)
        @test canallocate(batch, line)
    end

    @testset "Test prefers warehouse batches to shipments" begin
        tomorrow = Dates.today() + Dates.Day(1)
        warehousebatch = Batch(reference="warehouse-batch", sku="RETRO-CLOCK", qty=Qty(100))
        shipmentbatch  = Batch(reference="shipment-batch", sku="RETRO-CLOCK", qty=Qty(100), eta=tomorrow)
        line = OrderLine(reference="oref", sku="RETRO-CLOCK", qty=10)

        batchref = allocate_batches!(line, [warehousebatch; shipmentbatch])
        @test "warehouse-batch" == batchref
    end

    @testset "Test prefers earlier batches" begin
        today = Dates.today()
        tomorrow = today + Dates.Day(1)
        later = today + Dates.Day(30)

        earliest = Batch(reference="speedy-batch", sku="MINIMALIST-SPOON", qty=Qty(100), eta=today);
        medium   = Batch(reference="normal-batch", sku="MINIMALIST-SPOON", qty=Qty(100), eta=tomorrow);
        latest   = Batch(reference="slow-batch", sku="MINIMALIST-SPOON", qty=Qty(100), eta=later);
        line     = OrderLine(reference="order-001", sku="MINIMALIST-SPOON", qty=10)

        batchref = allocate_batches!(line, [medium; earliest; latest])
        
        @test "speedy-batch" == batchref
        @test 90 == availablequantity(earliest)
        @test 100 == availablequantity(medium)
        @test 100 == availablequantity(latest)
    end

    @testset "Test throws out of stock" begin
        today = Dates.today()
        batch = Batch(reference="batch1", sku="SMALFORK", qty=Qty(10), eta=today)
        line1 = OrderLine(reference="order-001", sku="MINIMALIST-SPOON", qty=10)
        line2 = OrderLine(reference="order-002", sku="SMALL-FORK", qty=1)
        allocate_batches!(line1, [ batch ])
        
        @test 0 == availablequantity(batch)
        @test !canallocate(batch, line2)
        @test_throws OutOfStock allocate_batches!(line2, [ batch ])
    end

end;

Test Summary:  | Pass  Total
Chapter1 Tests |   13     13
